# Clustering using K-means

In [1]:
from sklearn.cluster import KMeans, DBSCAN, AgglomerativeClustering
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import kneighbors_graph
from sklearn.metrics import silhouette_score, silhouette_samples

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from datetime import datetime
import time

In [2]:
data  = pd.read_csv('../Data/earthquake_dataset.csv')
data.head()

,Time,Latitude,Longitude,Depth/Km,Magnitude,EventLocationName
0,2010-10-13T21:41:46.570000,42.623,12.756,10.5,1.7,3 km W Ferentillo (TR)
1,2010-10-13T21:43:14.530000,42.457,13.390,10.8,1.7,8 km E Pizzoli (AQ)
2,2010-10-13T23:35:35.700000,42.470,13.377,11.0,0.8,8 km E Pizzoli (AQ)
3,2010-10-13T23:44:28.160000,42.474,13.393,12.9,1.3,9 km E Pizzoli (AQ)
4,2010-10-13T23:46:11.610000,42.448,13.387,10.2,2.0,8 km E Pizzoli (AQ)


In [3]:
dates = [datetime.strptime(x, '%Y-%m-%dT%H:%M:%S.%f') for x in data['Time']]
time_s = [time.mktime(d.timetuple()) / (60*60*24) for d in dates]  # days since epoch

data['Time/s'] = time_s

# select features
features = ['Latitude', 'Longitude', 'Depth/Km', 'Magnitude', 'Time/s'] # 'Year', 'Month', 'Day']#, 'Hour']
data = data[features]

data.head()

,Latitude,Longitude,Depth/Km,Magnitude,Time/s
0,42.623,12.756,10.5,1.7,14895.820671
1,42.457,13.390,10.8,1.7,14895.821690
2,42.470,13.377,11.0,0.8,14895.899711
3,42.474,13.393,12.9,1.3,14895.905880
4,42.448,13.387,10.2,2.0,14895.907072


In [4]:
features_clustering = ['Latitude', 'Longitude', 'Depth/Km', 'Magnitude']
X = data[features_clustering].values
#X = data.values

scaler = MinMaxScaler()
X = scaler.fit_transform(X)

X.shape

(29969, 4)

# K-means

In [ ]:
# choice of best value of  K that minimizes the sum of squared error
max_k = 30
sse_list = list([0] * (max_k-1))
silhouette_list = list([0] * (max_k-1))

for k in range(2, max_k + 1):
    print ("doing...", k)
    kmeans = KMeans(init='k-means++', n_clusters=k, n_init=5, max_iter=100, n_jobs=-2)
    kmeans.fit(X)
    # get sum of squared error
    sse_list[k-2] = kmeans.inertia_
    # approximate mean silhouette score
    trials = 5
    silho = 0
    for i in range(trials):
        silho += silhouette_score(X, kmeans.labels_, sample_size=8000)
    silhouette_list[k-2] = silho / trials

doing... 2
doing... 3


### choose optimal number of clusters

In [ ]:
# plot SSE and silhouette on the same scale

# first axis
fig, ax1 = plt.subplots(figsize=(10,5))
ax1.plot(range(2, max_k+1), sse_list, color='b', marker='o', linestyle='-', label='sum squared error', alpha=0.7)
ax1.set_xlabel('K')
ax1.set_ylabel('SSE', color='b')
ax1.tick_params('y', colors='b')
ax1.plot(np.nan, color='r',  marker='o', linestyle='--', label = 'silhouette score') # fake plot just to add single legend
ax1.legend(loc='best')

# second axis
ax2 = ax1.twinx()
ax2.plot(range(2, max_k+1), silhouette_list, color='red', marker='o', label='silhouette score', linestyle='--', alpha=0.7)
ax2.set_ylabel('Silhouette score', color='red')
ax2.tick_params('y', colors='red')


#plt.title('K-Means: SSE and Silhouette score vs K')
fig.tight_layout()
plt.savefig('../images/sse_silhouette_vs_k.png')
plt.show()

In [ ]:
chosen_k = 10
kmeans = KMeans(n_clusters=chosen_k)
kmeans.fit(X)

## Analysis of centroids

In [ ]:
markers = ['-', '--', '--*', ':', '--^', '-D', '-o', '-P']

plt.figure(1, figsize=(10, 3))
for i in range(0, len(kmeans.cluster_centers_)):
    plt.plot(range(0, X.shape[1]), kmeans.cluster_centers_[i], markers[i % len(markers)], 
             label='Cluster %s' % i, linewidth=2, markersize=11)

    plt.xticks(range(0, X.shape[1]), list(data.columns))

plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.show()

## Series of earthquake types

In [ ]:
labels = kmeans.labels_[-200:]
plt.figure(figsize=(20,5))
plt.step(range(len(labels)), labels, marker='o', alpha=0.7)
plt.ylabel('earthquake type')
plt.xlabel('sample')
plt.show()